In [24]:
import pandas as pd
import os

In [25]:
df=pd.read_excel('ami.xlsx')

In [26]:
df = df[df['AMI Categories'].notna()]

In [27]:
a=df.groupby(['Neighborhood','AMI Categories'])['Number of Units Tracked'].sum().reset_index()
a["AMI Categories"]=a["AMI Categories"].astype('string')

In [28]:
s = a['AMI Categories'].str.split(",|-").apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1)
del a['AMI Categories']
s.name = 'AMI Categories'
c=a.join(s)

In [29]:
def to_num(x):
    if '%' in x:
        return float(x.strip('%'))/100
    if float(x)>2:
        return float(x)/100
    return x
c['AMI Categories']=c['AMI Categories'].map(to_num)


In [30]:
c=c.groupby(['Neighborhood','AMI Categories'])['Number of Units Tracked'].sum().reset_index()

In [31]:
dic={'Allston':(42.355509,-71.132825),'Back Bay':(42.350265,-71.080976),'Brighton':(42.34635,-71.162676),'Charlestown':(42.3782065,-71.0602131),
'Dorchester':(42.30163,-71.067605),'Downtown':(42.355709,-71.057205),'East Boston':(42.370214,-71.03886),'Fenway':(42.342865,-71.100288),'Fort Point':(42.347099,-71.052005),'Jamaica Plain':(42.3097365,-71.11514310000001)
,'Mission Hill':(42.329617,-71.10623),'North End':(42.364702,-71.054234),'Roslindale':(42.2832142,-71.12702680000001),'Roxbury':(42.315198,-71.091375),'Seaport':(42.346252,-71.042144),'Mission Hill':(42.329617,-71.10623),
'South Boston':(42.3381437,-71.0475773),'South End':(42.338811,-71.076519),'West End':(42.365173,-71.064137),'West Roxbury':(42.279755,-71.162676),'Hyde Park':(42.256529,-71.124056)}

In [32]:
c['latitude']=c.Neighborhood.apply(lambda x:dic[x][0])
c['longitude']=c.Neighborhood.apply(lambda x:dic[x][1])

In [33]:
c['AMI Categories']=c['AMI Categories'].astype('float')

In [34]:
from folium import plugins
from folium.plugins import HeatMap
import folium
import numpy as np

In [53]:
c

,Neighborhood,AMI Categories,Number of Units Tracked,latitude,longitude
0,Allston,0.80,2.0,42.355509,-71.132825
1,Allston,1.00,4.0,42.355509,-71.132825
2,Allston,0.09,8.0,42.355509,-71.132825
3,Allston,0.70,122.0,42.355509,-71.132825
4,Back Bay,0.65,120.0,42.350265,-71.080976
...,...,...,...,...,...
71,West End,0.80,60.0,42.365173,-71.064137
72,West End,1.20,30.0,42.365173,-71.064137
73,West End,0.90,19.0,42.365173,-71.064137
74,West Roxbury,0.70,15.0,42.279755,-71.162676


In [58]:
d=c.loc[c['AMI Categories']>=1]

In [60]:
plot=d[['latitude','longitude','Number of Units Tracked']]

In [56]:
m = folium.Map(location=[42.3600825, -71.0588801] )
HeatMap(plot.values).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))

In [63]:
m